### Spark Session

Entry point to PySpark's functionality within a program. 

In [1]:
### Building SparkSession

from pyspark.sql import SparkSession   ### SparkSession entry point located in pyspark.sql pkg, providing functionality for data transformation

spark = (SparkSession
         .builder     ### Builder pattern abstraction for constructing a sparksession, where we chain the methods to configure the entry point.
         .appName("Analyzing the vocabulary of Pride and Prejudice.") ### Relevant appName helping in identifying which programs run on the Spark cluster
         .getOrCreate()) ### Program works in both interactive or batch mode by avoiding creating a new session if one already exists.

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/08 23:28:51 WARN Utils: Your hostname, OnePiece, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/08 23:28:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/08 23:28:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sparkContext ### For using low-level RDD

""" SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD"""



' SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD'

### Setting the Log Level

Monitoring PySpark jobs is an important part of developing. PySpark has many levels of logging, from nothing to full description of everything happening in cluster. pyspark shell defaults on WARN, which is a bit chatty.  And, PySpark program defaults to INFO level which is oversharing. 

In [3]:
### Changing the log level to keyword

spark.sparkContext.setLogLevel("KEYWORD")

### Keywords: WARN, ALL, INFO, DEBUG, TRACE, OFF, etc.

IllegalArgumentException: requirement failed: Supplied level KEYWORD did not match one of: ALL,DEBUG,ERROR,WARN,INFO,OFF,FATAL,TRACE

In [4]:
dir(spark.read)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_df',
 '_jreader',
 '_set_opts',
 '_spark',
 'csv',
 'format',
 'jdbc',
 'json',
 'load',
 'option',
 'options',
 'orc',
 'parquet',
 'schema',
 'table',
 'text',
 'xml']

In [5]:
#### Reading the csv file

book = spark.read.text("./gutenberg.txt")

book

DataFrame[value: string]

In [6]:
## To get the schema of the dataframe

book.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
### For documentation use
print(spark.__doc__)

The entry point to programming Spark with the Dataset and DataFrame API.

    A SparkSession can be used to create :class:`DataFrame`, register :class:`DataFrame` as
    tables, execute SQL over tables, cache tables, and read parquet files.
    To create a :class:`SparkSession`, use the following builder pattern:

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    .. autoattribute:: builder
       :annotation:

    Examples
    --------
    Create a Spark session.

    >>> spark = (
    ...     SparkSession.builder
    ...         .master("local")
    ...         .appName("Word Count")
    ...         .config("spark.some.config.option", "some-value")
    ...         .getOrCreate()
    ... )

    Create a Spark session with Spark Connect.

    >>> spark = (
    ...     SparkSession.builder
    ...         .remote("sc://localhost")
    ...         .appName("Word Count")
    ...         .config("spark.some.config.option", "some-value")
    ...         .getOrCreate()
    ..

In [9]:
# To see the contents of the dataframe

book.show(n=10, truncate = False, vertical = True) # Shows 20 rows and truncates long values.
## n = no.of rows, truncate = default truncates at 20 chars, vertical = displays each record as a small table

-RECORD 0---------------------------------------------------------------------
 value | The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen   
-RECORD 1---------------------------------------------------------------------
 value |                                                                      
-RECORD 2---------------------------------------------------------------------
 value | This eBook is for the use of anyone anywhere at no cost and with     
-RECORD 3---------------------------------------------------------------------
 value | almost no restrictions whatsoever.  You may copy it, give it away or 
-RECORD 4---------------------------------------------------------------------
 value | re-use it under the terms of the Project Gutenberg License included  
-RECORD 5---------------------------------------------------------------------
 value | with this eBook or online at www.gutenberg.org                       
-RECORD 6-------------------------------------------

In [ ]:
"""
Generally spark is lazily evaluated, but you want eager evaluation, similar to pandas you can change the mode

from pyspark.sql import SparkSession

spark = (SparkSession.builder
                     .config("spark.sql.repl.eagerEval.enabled", "True")
                     .getOrCreate())
"""

In [11]:
### Step-2 Tokenize the words 

#### We'll be splitting the lines of text into arrays or words

from pyspark.sql.functions import split

lines = book.select(split(book.value," ").alias("line")) ## Each record is stored in value. Splitting with space as separator

lines. show(5)

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows


In [12]:
### Otherways to select a value column from the dataframe

from pyspark.sql.functions import col

book.select(book.value)
book.select(book["value"]) # Use this when the col names contains any special chars
book.select(col["value"]) # No need to mention the dataframe
book.select("value") # Might become problematic in case of transformations

DataFrame[value: string]

In [14]:
### Transforming columns: Splitting a string into a list of words

from pyspark.sql.functions import col, split

lines = book.select(split(col("value"), " "))

lines.printSchema()

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)



In [16]:
lines.show(5) ## [] represents array, if its empty it means there are no values present or that row is empty

+--------------------+
| split(value,  , -1)|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows


In [17]:
### Aliasing 
book.select(split(col("value"), " ").alias("line")). printSchema() ###You can use .withColumnRenamed method as well to alias


root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [20]:
lines = book.select(split(book.value, " "))
lines = lines.withColumnRenamed("split(value, , -1)", "line")

In [24]:
lines = book.select(split(book.value, " ").alias("line"))

In [25]:
#### Reshaping the data: Exploding a list into Rows

"""After splitting the records, we have arrays of strings and it would be better to have one record for each word"""

from pyspark.sql.functions import explode, col

words = lines.select(explode(col("line")).alias("word"))

words.show(15)
                                                

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     EBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
+----------+
only showing top 15 rows


In [26]:
### Step-3 Removing punctuation and turning into lower case

from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))

words_lower.show()

[Stage 5:>                                                          (0 + 1) / 1]

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows


In [29]:
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]+" , 0).alias("word")) # We only match for multiple lowercase chars, + will match for one or more occurences

words_clean.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
+---------+
only showing top 20 rows


In [ ]:
### Filtering Rows

words_nonnull = words_clean.filter